In [1]:
%%time
%matplotlib inline
import pandas as pd
import re
from tool import import_dir; import_dir("../")
from src.lib import nlp as libnlp

CPU times: user 988 ms, sys: 225 ms, total: 1.21 s
Wall time: 1.26 s


# Dataset

In [2]:
%time data = pd.read_csv("../data/rdt-050409-111819.csv")

CPU times: user 154 ms, sys: 19.9 ms, total: 174 ms
Wall time: 191 ms


# PreProcess Data

In [4]:
%%time
data = data.dropna()
data.index = pd.to_datetime(data.created_at)
data = data.drop(["created_at"], axis="columns")

CPU times: user 4.59 s, sys: 2.13 ms, total: 4.59 s
Wall time: 4.61 s


# Filter
This drops tweets that start with links under the assumption that the body only contains a link.

In [5]:
%%time
f = lambda x: None if x[:4] in "http" else x
data.text = data.text.apply(f)
data = data.dropna()
data = data[data.index > "2019-02"]

CPU times: user 54.8 ms, sys: 1.94 ms, total: 56.7 ms
Wall time: 56.1 ms


# Language Model

In [3]:
%time (nlp, keys, vectors) = libnlp.load_model("en_core_web_lg")

CPU times: user 8.73 s, sys: 1.15 s, total: 9.88 s
Wall time: 10.9 s


# Parse text with language model

In [14]:
f = lambda x: nlp(x)
%time data.text = data.text.apply(f)

CPU times: user 1min 2s, sys: 129 ms, total: 1min 2s
Wall time: 1min 2s


# Tokenize

In [12]:
%time tokenizer = libnlp.tokenizer(nlp)

CPU times: user 19.9 s, sys: 6 s, total: 25.9 s
Wall time: 26.1 s


In [15]:
f = lambda corpra: libnlp.tokenize(tokenizer, corpra)
%time data["tokens"] = data.text.apply(f)

CPU times: user 2.26 s, sys: 27.9 ms, total: 2.29 s
Wall time: 2.3 s


# Derive Topic

In [17]:
f = lambda tokens: libnlp.topic_vector(nlp, tokens)
%time data["topic_vector"] = data.tokens.apply(f)

CPU times: user 1.29 s, sys: 5.11 ms, total: 1.3 s
Wall time: 1.3 s


In [19]:
f = lambda vector: libnlp.topic_string(nlp, keys, vectors, vector)
%time data["topic"] = data.topic_vector.apply(f)

CPU times: user 1h 31min 18s, sys: 1h 6min 23s, total: 2h 37min 41s
Wall time: 2h 38min 26s


In [21]:
data.head()

,source,text,retweet_count,favorite_count,is_retweet,id_str,tokens,topic_vector,topic
created_at,,,,,,,,,
2019-11-18 03:39:38,Twitter for iPhone,"(“, All, they, do, is, bring, up, witnesses, w...",16083.0,60217,false,1.196272e+18,"[“, All, they, do, is, bring, up, witnesses, w...","[-0.06569162, 0.049224027, -0.13754201, -0.038...",ThaT
2019-11-18 02:40:04,Twitter for iPhone,"(“, The, Impeachment, started, before, he, eve...",18078.0,73353,false,1.196257e+18,"[“, The, Impeachment, started, before, he, eve...","[0.025502997, 0.089400284, -0.02185314, -0.081...",WHen
2019-11-17 20:12:27,Twitter for iPhone,"(Where, is, the, Fake, Whistleblower, ?)",26145.0,122225,false,1.196159e+18,"[Where, is, the, Fake, Whistleblower, ?]","[-0.13752584, 0.074059494, -0.18962795, -0.041...",ThaT
2019-11-17 20:08:08,Twitter for iPhone,"(Republicans, &, amp, ;, others, must, remembe...",16764.0,62783,false,1.196158e+18,"[Republicans, &, amp, ;, others, must, remembe...","[-0.123433076, 0.16298212, -0.110808805, -0.12...",THERE
2019-11-17 20:03:44,Twitter for iPhone,"(The, Crazed, Do, Nothing, Democrats, are, tur...",17646.0,69248,false,1.196157e+18,"[The, Crazed, Do, Nothing, Democrats, are, tur...","[-0.10330248, 0.112459324, -0.016367562, -0.06...",way


# Save

In [22]:
%time data.to_csv("../data/rdt_topic_201902_201911.csv")

CPU times: user 15.3 s, sys: 57.9 ms, total: 15.4 s
Wall time: 15.5 s
